In [1]:
import os
import pandas as pd
import numpy as np
import datetime

import plotly.graph_objects as go
from plotly.subplots import make_subplots

%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

import chart_utils
import analysis_utils

In [2]:
# # Download CoinMetrics data if needed

# try:
#     os.remove("btc.csv")
# except FileNotFoundError:
#     pass
# !wget https://coinmetrics.io/newdata/btc.csv

In [32]:
data = pd.read_csv("data/05_coinbase_herfindahl_daily.csv")
data = analysis_utils.get_extra_datetime_cols(data, 'coinbase_block_ts', date_format="%Y-%m-%d %H:%M:%S")
price_data = pd.read_csv("btc.csv")
data = data.merge(price_data[
    ['date', 'PriceUSD']], on='date')
data.rename(columns = {'herfindal_index':'herfindahl_index'}, inplace = True)

In [33]:
daily_data = data.groupby('date', as_index=False).mean()
daily_data

,date,coinbase_block_number,coinbase_total_value,herfindahl_index,PriceUSD
0,2009-01-03,0.0,50.000000,1.000000,NaN
1,2009-01-09,7.5,50.000000,0.945197,NaN
2,2009-01-10,45.0,50.000000,1.000000,NaN
3,2009-01-11,122.0,49.999996,0.991597,NaN
4,2009-01-12,215.5,50.000000,1.000000,NaN
...,...,...,...,...,...
4282,2020-09-29,650505.0,6.835730,0.355296,10841.142744
4283,2020-09-30,650653.5,6.856768,0.481647,10772.351767
4284,2020-10-01,650793.5,7.071466,0.636753,10606.617333
4285,2020-10-02,650923.5,7.150505,0.797951,10570.105678


In [38]:
window_sizes = [1, 7, 14, 28, 56, 70]
for window_size in window_sizes:
    daily_data['herfindahl_index_{}_roll'.format(window_size)] = daily_data['herfindahl_index'].rolling(window_size).mean()
    daily_data['herfindahl_index_multiple_{}_roll'.format(window_size)] = daily_data['herfindahl_index'].rolling(window_size).mean() / daily_data['herfindahl_index'].rolling(365).mean()

In [35]:
chart_utils.two_axis_chart(
    daily_data[:-28], x_series='date', 
    y1_series='herfindahl_index_28_roll',
    y2_series='PriceUSD',
    y2_series_axis_type='log', y2_series_axis_range=[-2, 6],
    x_axis_title='Date',
    y1_series_title='Herfindahl Index',
#     y1_series_axis_format='{:.2f}',
    title='Coinbase Output Herfindahl Index',
    data_source="Proprietary; Price from CoinMetrics",
    halving_lines=True
)

In [41]:
chart_utils.two_axis_chart(
    daily_data[:-28], x_series='date', 
    y1_series='herfindahl_index_multiple_28_roll',
    y2_series='PriceUSD',
    y2_series_axis_type='log', y2_series_axis_range=[-2, 6],
    x_axis_title='Date',
    y1_series_title='Herfindahl Index Multiple',
    title='Coinbase Output Herfindahl Index Multiple',
    data_source="Proprietary; Price from CoinMetrics",
    halving_lines=True,
    save_file='./docs/MinerHerfMultiple.html'
)

In [12]:
curves = pd.read_csv("data/05_miner_herf_4_curve.csv")
curves = analysis_utils.get_extra_datetime_cols(curves, 'date', date_format="%Y-%m-%dT%H:%M:%S")
curves[:5]

,date,days_since_coinbase,herfindal_index,datetime,year_string,month_string,week_string
0,2009-01-01,0,1.000000,2009-01-01,2009,January 2009,2008-12-29
1,2009-01-01,1,1.000000,2009-01-01,2009,January 2009,2008-12-29
2,2009-01-01,2,0.999586,2009-01-01,2009,January 2009,2008-12-29
3,2009-01-01,3,0.999348,2009-01-01,2009,January 2009,2008-12-29
4,2009-01-01,4,0.999348,2009-01-01,2009,January 2009,2008-12-29


In [13]:
chart_utils.miner_herf_chart(
    curves.loc[curves['date'] == '2020-05-01'].reset_index(drop=True),
    y1_series_title='Coinbase Output Herfindahl Index',
    y1_series_axis_type='log',
    y1_series_axis_range=[-1,0],
#     y1_series_axis_format='{:0.1f}',
    title='Coinbase Output Herfindahl Index Curve: 4 deep',
    x_axis_title='Days since mined',
    data_source="Proprietary"
)

In [20]:
chart_utils.miner_herf_chart(
    curves.loc[curves['date'] >= '2020-01-01'].reset_index(drop=True),
    y1_series_title='Coinbase Output Herfindahl Index',
    y1_series_axis_type='linear',#'log',
    y1_series_axis_range=[0, 1],#[-1,0],
#     y1_series_axis_format='{:0.1f}',
    title='Coinbase Output Herfindahl Index Curve',
    x_axis_title='Days since mined',
    data_source="Proprietary"
)

In [15]:
curves_halving = pd.read_csv("data/05_miner_herf_4_curve_halvings.csv")
curves_halving[:5]

,period,days_since_coinbase,herfindal_index
0,First halving: 3 months following,0,0.956511
1,First halving: 3 months following,1,0.686177
2,First halving: 3 months following,2,0.620068
3,First halving: 3 months following,3,0.599245
4,First halving: 3 months following,4,0.585519


In [19]:
chart_utils.miner_herf_chart(
    curves_halving,
    y1_series_title='Coinbase Output Herfindahl Index',
    y1_series_axis_type='linear',#'log',
    y1_series_axis_range=[0, 1],#[-1,0],
#     y1_series_axis_format='{:0.1f}',
    title='Coinbase Output Herfindahl Index Curve',
    x_axis_title='Days since coinbase mined',
    data_source="Proprietary",
    pivot='period'
)